#**TEXT CLASSIFICATION**
 - Text preprocessing – stopwords removal and porter stemming algorithm
 - Word embeddings using keras
 - Padding the sequence
 - Fix the vocabulary size
 - One hot encoding
 - Construct the sequential model RNN and LSTM
 - Evaluate with accuracy score


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Input
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### **TEXT PRE-PROCESSING - STOPWORDS REMOVAL AND PORTER STEMMING ALGORITHM**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

In [ ]:
def preprocess_text(text):
    tokens = text.split()
    tokens = [porter.stem(word) for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-08-17 17:15:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-17 17:15:52 (78.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
unzip_data("nlp_getting_started.zip")
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

--2024-08-17 17:15:52--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.174.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   930KB/s    in 0.6s    

2024-08-17 17:15:53 (930 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42)

In [ ]:
train_sentences = [preprocess_text(sentence) for sentence in train_sentences]
val_sentences = [preprocess_text(sentence) for sentence in val_sentences]

### **WORD EMBEDDINGS USING KERAS**

In [ ]:
max_vocab_length = 10000
max_length = 15
embedding_dim = 16

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_vocab_length, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

### **PADDING THE SEQUENCE**

In [ ]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
padded_val_sequences = pad_sequences(val_sequences, maxlen=max_length, padding='post', truncating='post')

###**CONSTRUCT THE SEQUENTAIL MODEL**

In [ ]:
inputs = Input(shape=(max_length,))
x = Embedding(input_dim=max_vocab_length, output_dim=embedding_dim, input_length=max_length)(inputs)
x = LSTM(64)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="text_classification_model")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "text_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 15)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 16)              │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          20,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 180,801 (706.25 KB)

 Trainable params: 180,801 (706.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_history = model.fit(padded_train_sequences,
                          train_labels,
                          epochs=5,
                          validation_data=(padded_val_sequences, val_labels),
                          callbacks=[create_tensorboard_callback("model_logs", "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20240817-171601
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6696 - loss: 0.6036 - val_accuracy: 0.7703 - val_loss: 0.4686
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8583 - loss: 0.3418 - val_accuracy: 0.7835 - val_loss: 0.4741
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9134 - loss: 0.2295 - val_accuracy: 0.7743 - val_loss: 0.5186
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9365 - loss: 0.1741 - val_accuracy: 0.7612 - val_loss: 0.6599
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9529 - loss: 0.1415 - val_accuracy: 0.7690 - val_loss: 0.5995


In [ ]:
model_pred_probs = model.predict(padded_val_sequences)
model_preds = tf.squeeze(tf.round(model_pred_probs))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


###**EVALUATE THE ACCURACY SCORE**

In [ ]:
def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [ ]:
model_results = calculate_results(y_true=val_labels, y_pred=model_preds)
print(model_results)

{'accuracy': 76.9028871391076, 'precision': 0.7798518825968084, 'recall': 0.7690288713910761, 'f1': 0.7634085385385204}
